In [ ]:
# 1、找出136套数据所有细胞类型的markers（1vn）
suppressWarnings(library(Seurat))

rawdata_dir = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/2.tisch_count_data2"
dataset_markers_dir = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/2.tisch_data/6.Marker/DatasetMarker"
rawdata_list = list.files(rawdata_dir, pattern = "\\.csv$")
for (i in 1:length(rawdata_list)) {
    rawdata_filename = rawdata_list[i]
    print(paste0(rawdata_filename, " process start."))
    markers_filename = paste0(dataset_markers_dir, "/", rawdata_filename)
    if (file.exists(markers_filename)) {
        print(paste0(markers_filename, " exists."))
        next
    }

    rawdata_path = paste0(rawdata_dir, "/", rawdata_filename)
    rawdata_df = read.delim(file=rawdata_path, header = TRUE, sep = ",", row.names = 1, check.names=FALSE)
    seurat = CreateSeuratObject(counts = rawdata_df, names.field = 1, names.delim = '\\.')
    result = try({
        immune_seurat = subset(seurat, orig.ident %in% c("CD8T", "Mono/Macro", "Neutrophils", "NK", "B"))
    }, silent = TRUE)
    if (inherits(result, "try-error")) {
        print(paste0(rawdata_filename, " has no immune cell."))
        next
    }
    
    immune_seurat = NormalizeData(immune_seurat)
    # all cell type: positive markers
    dataset_markers <- FindAllMarkers(immune_seurat, only.pos=T)
    if (length(dataset_markers) > 0) {
        write.table(dataset_markers[c(6,7,2,3,4,1,5)], file=markers_filename, row.names=F, col.names=T, sep=',', quote=F)
    }
    print(paste0(rawdata_filename, " process end."))
}

In [58]:
import numpy as np
import pandas as pd
import os

dataset_markers_dir = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/2.tisch_data/6.Marker/DatasetMarker2"
celltype_markers_dir = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/2.tisch_data/6.Marker/CelltypeMarker"

all_markers = {}
celltype_filenum = {}
dataset_markers_files = sorted(os.listdir(dataset_markers_dir))
for dataset_markers_file in dataset_markers_files:
    dataset_markers_path = os.path.join(dataset_markers_dir, dataset_markers_file)
    dataset_markers = pd.read_csv(dataset_markers_path, index_col=0, header=0)
    dataset_celltype = dataset_markers.index.unique().tolist()
    for celltype in dataset_celltype: # 统计每个celltype的
        celltype = celltype.strip().replace("/", "_")
        if celltype == "Cholangiocyte": celltype = "Cholangiocytes"
        elif celltype == "Fibroblast": celltype = "Fibroblasts"
        elif celltype == "Hepatocyte": celltype = "Hepatocytes"
        elif celltype == "Mono": celltype = "Mono_Macro"
        elif celltype == "TProlif": celltype = "Tprolif"
        celltype_filenum[celltype] = celltype_filenum.get(celltype, 0) + 1

    for celltype, row in dataset_markers.iterrows():
        celltype = celltype.strip().replace("/", "_")
        if celltype == "Cholangiocyte": celltype = "Cholangiocytes"
        elif celltype == "Fibroblast": celltype = "Fibroblasts"
        elif celltype == "Hepatocyte": celltype = "Hepatocytes"
        elif celltype == "Mono": celltype = "Mono_Macro"
        elif celltype == "TProlif": celltype = "Tprolif"
        if celltype not in all_markers.keys():
            new_dataset_markers = pd.DataFrame(columns=['gene', 'include_num'])
            new_row = {'gene': row['gene'], 'include_num': 1}
            new_dataset_markers.loc[len(new_dataset_markers)] = new_row
            all_markers[celltype] = new_dataset_markers
        else:
            old_dataset_markers = all_markers[celltype]
            if row['gene'] not in list(old_dataset_markers['gene']):
                new_row = {'gene': row['gene'], 'include_num': 1}
                old_dataset_markers.loc[len(old_dataset_markers)] = new_row
            else:
                index = old_dataset_markers[old_dataset_markers['gene'] == row['gene']].index.values[0]
                old_dataset_markers.loc[index, 'include_num'] += 1
    print(f'{dataset_markers_file} process end.')
for celltype, markers in all_markers.items():
    markers['dataset_num'] = markers['gene'].apply(lambda x: celltype_filenum[celltype])
    markers['rate'] = markers['include_num'] / markers['dataset_num']
    markers.to_csv(os.path.join(celltype_markers_dir, f'{celltype}.csv'))

AEL_GSE142213.csv process end.
ALL_GSE132509.csv process end.
ALL_GSE153697.csv process end.
ALL_GSE154109.csv process end.
AML_GSE116256.csv process end.
AML_GSE135851.csv process end.
AML_GSE154109.csv process end.
BCC_GSE123813_aPD1.csv process end.
BCC_GSE141526.csv process end.
BLCA_GSE130001.csv process end.
BRCA_GSE110686.csv process end.
BRCA_GSE114727_10X.csv process end.
BRCA_GSE114727_inDrop.csv process end.
BRCA_GSE136206_mouse_aPD1aCTLA4.csv process end.
BRCA_GSE143423.csv process end.
BRCA_GSE148673.csv process end.
BRCA_GSE150660.csv process end.
CESC_GSE168652.csv process end.
CHOL_GSE125449_aPD1aPDL1aCTLA4.csv process end.
CHOL_GSE138709.csv process end.
CHOL_GSE142784.csv process end.
CLL_GSE111014.csv process end.
CLL_GSE125881.csv process end.
CLL_GSE132065.csv process end.
CLL_GSE142744.csv process end.
CLL_GSE152469.csv process end.
CLL_GSE159417.csv process end.
CRC_EMTAB8107.csv process end.
CRC_GSE112865_mouse_aPD1.csv process end.
CRC_GSE122969_mouse_aPD1aTIM3

In [33]:
suppressWarnings(library(Seurat))
library(dplyr)

rawdata_path = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/rawdata"
rawdata = Read10X(data.dir = rawdata_path)
tissue <- c("N", "T", "T1", "T2", "LN", "TN")
col_names <- colnames(rawdata)
selected_col_names <- which(sapply(strsplit(col_names, "-"), function(x) x[2]) %in% tissue)
rawdata <- rawdata[, selected_col_names]

seurat_data <- CreateSeuratObject(counts = rawdata, project = "GSE236581", min.features = 600)

# 质控 Specifically, cells with (1) less than 600 or more than 25,000 UMI counts, (2) less than 600 detected genes, or
# (3) more than 5% (70% for CD45− cells) mitochondrial gene counts,were filtered out.
seurat_data[["percent.mt"]] <- PercentageFeatureSet(seurat_data, pattern = "^MT-")
seurat_data <- subset(seurat_data, subset = nCount_RNA > 600 & nCount_RNA < 25000 & percent.mt < 5)


seurat_data <- NormalizeData(seurat_data)
seurat_data <- ScaleData(seurat_data)
seurat_data <- FindVariableFeatures(seurat_data)
seurat_data <- RunPCA(seurat_data)
# seurat_data <- FindNeighbors(seurat_data, compute.SNN = FALSE, reduction = "pca", dims = 1:50, graph.name="nn")
# seurat_data <- FindClusters(seurat_data, algorithm = 1, resolution = 0.8, graph.name="nn")
seurat_data <- FindNeighbors(seurat_data, reduction = "pca", dims = 1:50)
seurat_data <- FindClusters(seurat_data, algorithm = 1, resolution = 0.8)

markers_filename <- "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/markers.csv"
markers <- FindAllMarkers(seurat_data, only.pos=T)
if (length(markers) > 0) {
    write.table(markers[c(6,7,2,3,4,1,5)], file=markers_filename, row.names=F, col.names=T, sep=',', quote=F)
}

# celltype_markers_path = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/2.tisch_data/6.Marker/CelltypeMarker/B.csv"
# celltype_markers <- read.csv(celltype_markers_path, row.names = 1)
# top_100_indices <- order(data[['rate']], decreasing = TRUE)[1:1000] 
# celltype_markers_top1000 <- data[top_100_indices, 'gene']
B_score = list(c("PTPRC", "CD79A", "CD79B", "MS4A1", "BANK1", "CD19"))
CD8T_score = list(c("PTPRC", "CD3E", "CD3D", "CD3G", "CD8B", "CD8A", "CCL5", "GZMK"))
Mono_Macro_score = list(c("TPRC", "C1QA", "C1QB", "CD14", "CD86", "CD68", "CTSS", "TGFBI", "LYZ", "CSF1R", "IL1B", "CD300E", "S100A12"))
Neutrophils_score = list(c("PTPRC", "S100A8", "S100A9"))
NK_score = list(c("PTPRC", "NCAM1", "FCGR3A", "KLRF1", "KLRD1"))
Immune_score = list(c("PTPRC"))
pDC_score = list(c("PTPRC", "LILRA4", "IL3RA", "CLEC4C"))
DC_score = list(c("PTPRC", "CD1C", "CD209"))
T_score = list(c("PTPRC", "CD3E", "CD3D", "CD3G"))
CD4T_score = list(c("PTPRC", "CD3E", "CD3D", "CD3G", "CD4", "IL7R"))
Epithelial_score = list(c("EPCAM"))
Vascular_score = list(c("PECAM1", "PLVAP", "RGS5"))
Stromal_score = list(c("DCN", "ACTA2", "MYH11", "PDGFRA"))
Mast_score = list(c("TPSAB1", "TPBS2"))

seurat_data =  AddModuleScore(object = seurat_data, features = B_score, name = "B_score")
seurat_data =  AddModuleScore(object = seurat_data, features = CD8T_score, name = "CD8T_score")
seurat_data =  AddModuleScore(object = seurat_data, features = Mono_Macro_score, name = "Mono_Macro_score")
seurat_data =  AddModuleScore(object = seurat_data, features = Neutrophils_score, name = "Neutrophils_score")
seurat_data =  AddModuleScore(object = seurat_data, features = NK_score, name = "NK_score")
seurat_data =  AddModuleScore(object = seurat_data, features = Immune_score, name = "Immune_score")
seurat_data =  AddModuleScore(object = seurat_data, features = pDC_score, name = "pDC_score")
seurat_data =  AddModuleScore(object = seurat_data, features = DC_score, name = "DC_score")
seurat_data =  AddModuleScore(object = seurat_data, features = T_score, name = "T_score")
seurat_data =  AddModuleScore(object = seurat_data, features = CD4T_score, name = "CD4T_score")
seurat_data =  AddModuleScore(object = seurat_data, features = Epithelial_score, name = "Epithelial_score")
seurat_data =  AddModuleScore(object = seurat_data, features = Vascular_score, name = "Vascular_score")
seurat_data =  AddModuleScore(object = seurat_data, features = Stromal_score, name = "Stromal_score")
seurat_data =  AddModuleScore(object = seurat_data, features = Mast_score, name = "Mast_score")

saveRDS(seurat_data, file = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/seurat_data.rds")

# 提取meta数据
score_data <- seurat_data@meta.data %>% select(seurat_clusters, B_score1, CD8T_score1, Mono_Macro_score1, Neutrophils_score1, NK_score1, Immune_score1, pDC_score1, DC_score1, T_score1, CD4T_score1, Epithelial_score1, Vascular_score1, Stromal_score1, Mast_score1)
score_filename = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/cell_score.csv"
write.table(score_data, file=score_filename, row.names=F, col.names=T, sep=',', quote=F)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3732528398.py, line 4)

In [60]:
import numpy as np
import pandas as pd
import os

celltype_markers_dir = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/2.tisch_data/6.Marker/CelltypeMarker"
celltype_list = []
celltype_markers_list = os.listdir(celltype_markers_dir)
for filename in celltype_markers_list:
    celltype_list.append(filename.split(".")[0])
all_cluster_markers_path = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/markers.csv"
result = pd.DataFrame(columns=celltype_list)
for i in range(40):
    new_row = {}
    for celltype in celltype_list:
        celltype_markers = pd.read_csv(os.path.join(celltype_markers_dir, f"{celltype}.csv"), index_col=0, header=0)
        celltype_markers_top2000 = celltype_markers.nlargest(n=200, columns='rate')['gene'].tolist()

        all_cluster_markers = pd.read_csv(all_cluster_markers_path, index_col=0, header=0)
        cluster_markers = all_cluster_markers[all_cluster_markers.index == i]['gene'].tolist()
        intersection = set(celltype_markers_top2000).intersection(set(cluster_markers))
        intersection_rate = len(intersection) / len(celltype_markers_top2000)
        new_row[celltype] = intersection_rate
    #     print("len(all_cluster_markers)", len(cluster_markers), "len(celltype_markers_top2000)", len(celltype_markers_top2000), "intersection", len(intersection))
    print(f"Cluster {i} process end.")
    result.loc[len(result)] = new_row
result.to_csv("/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/celltype_cluster_map.csv")

Cluster 0 process end.
Cluster 1 process end.
Cluster 2 process end.
Cluster 3 process end.
Cluster 4 process end.
Cluster 5 process end.
Cluster 6 process end.
Cluster 7 process end.
Cluster 8 process end.
Cluster 9 process end.
Cluster 10 process end.
Cluster 11 process end.
Cluster 12 process end.
Cluster 13 process end.
Cluster 14 process end.
Cluster 15 process end.
Cluster 16 process end.
Cluster 17 process end.
Cluster 18 process end.
Cluster 19 process end.
Cluster 20 process end.
Cluster 21 process end.
Cluster 22 process end.
Cluster 23 process end.
Cluster 24 process end.
Cluster 25 process end.
Cluster 26 process end.
Cluster 27 process end.
Cluster 28 process end.
Cluster 29 process end.
Cluster 30 process end.
Cluster 31 process end.
Cluster 32 process end.
Cluster 33 process end.
Cluster 34 process end.
Cluster 35 process end.
Cluster 36 process end.
Cluster 37 process end.
Cluster 38 process end.
Cluster 39 process end.


In [55]:
import numpy as np
import pandas as pd
import os

cell_score_path = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/cell_score.csv"
cell_score = pd.read_csv(cell_score_path, index_col = 0, header = 0)

result = pd.DataFrame()
for col in cell_score.columns.values:
    cell_score_celltype = cell_score[col]
    q75 = cell_score_celltype.quantile(0.75)
    cell_score_celltype_grouped = cell_score_celltype.groupby(level=0)
    for group, groupdata in cell_score_celltype_grouped:
        rate = np.mean(groupdata > q75)
        result.loc[group, col] = rate
result_path = "/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581/cluster_score.csv"
result.to_csv(result_path)

In [ ]:
setwd("/sibcb1/bioinformatics/hongyuyang/dataset/Tres/4.analysis_data/sc/GSE236581")
suppressWarnings(library(Seurat))
library(ggplot2)

seurat_data = readRDS("seurat_data.rds")
seurat_data <- RunUMAP(seurat_data, reduction = "pca", dims = 1:50)

all_markers = c("nCount_RNA", "nFeature_RNA", "PTPRC", "CD79A", "CD79B", "MS4A1", "BANK1", "CD19", "PTPRC", "CD3E", "CD3D", "CD3G", "CD8B", "CD8A", "CCL5", "GZMK", "TPRC", "C1QA", "C1QB", "CD14", "CD86", "CD68", "CTSS", "TGFBI", "LYZ", "CSF1R", "IL1B", "CD300E", "S100A12", "PTPRC", "S100A8", "S100A9", "PTPRC", "NCAM1", "FCGR3A", "KLRF1", "KLRD1", "PTPRC", "LILRA4", "IL3RA", "CLEC4C", "PTPRC", "CD1C", "CD209", "PTPRC", "CD3E", "CD3D", "CD3G", "PTPRC", "CD3E", "CD3D", "CD3G", "CD4", "IL7R", "EPCAM", "PECAM1", "PLVAP", "RGS5", "DCN", "ACTA2", "MYH11", "PDGFRA", "TPSAB1", "TPBS2", "MKI67", "MS4A1", "CD79A", "CD3D", "IL7R", "CCR7", "S100A4", "CD8A", "GNLY", "NKG7", "CD14", "LYZ", "FCGR3A", "MS4A7", "FCER1A", "CST3", "PPBP")
colors <- c('#1C79B7','#F38329','#2DA248','#DC403E','#976BA6','#8D574C','#D07DB0','#BDBF3B', '#27BDD0','#B0C7E5','#F9BA79','#A0CC8A','#F09EA1','#C7B0D1','#D4BCB8','#F7CCDD',
            '#DBDD8D','#A3D7E5','#B04E4F','#A38A58','#ED5351','#0C8945','#0F71A7','#A82764', '#F8DAE4','#7A4B1F','#5E6BAE','#8AC997','#DAC9AC','#0F5148','#A0B5DC','#9F858E',
            '#5C181D','#7B8380','#E8DDDA','#264220','#5AB747','#5169AE','#4B3D58','#CD428A', '#62615A','#B82129','#66762E')

plot1 <- DimPlot(seurat_data, cols=colors, label=TRUE)
ggsave(plot1, file='DimPlot.pdf', width = 10, height = 10)

plot1 <- FeaturePlot(seurat_data, features = unique(all_markers), cols=c('grey95', 'red3'))
ggsave(plot1, file='FeaturePlot_all.pdf', width = 20, height = 50, limitsize = FALSE)

plot1 <- VlnPlot(seurat_data, features = unique(all_markers), cols=colors) + NoLegend()
ggsave(plot1, file='VlnPlot_all.pdf', width = 50, height = 30, limitsize = FALSE)

plot1 <- DotPlot(seurat_data, features = unique(all_markers)) + theme_bw() + RotatedAxis() + coord_flip() + scale_color_gradient(low = "grey95", high = 'Red3') + theme(panel.background = element_blank(), axis.title = element_blank(),axis.text = element_text(size = 12), legend.title = element_text(size = 12), legend.text = element_text(size = 12))
ggsave(plot1, file='DotPlot_all.pdf', width = 30, height = 20, limitsize = FALSE)

seurat_data@meta.data$annotation = 'other' 
seurat_data@meta.data[seurat_data$seurat_clusters %in% c('2', '4', '8', '29'), 'annotation'] = 'CD8T'
seurat_data@meta.data[seurat_data$seurat_clusters %in% c('28'), 'annotation'] = 'Mono_Macro'
seurat_data@meta.data[seurat_data$seurat_clusters %in% c('13'), 'annotation'] = 'Neutrophils'
seurat_data@meta.data[seurat_data$seurat_clusters %in% c('15', '16'), 'annotation'] = 'NK'
seurat_data@meta.data[seurat_data$seurat_clusters %in% c('3'), 'annotation'] = 'B'

saveRDS(seurat_data, file = "all_seurat.rds")

seurat_CD8T <- subset(seurat_data, cells = rownames(seurat_data@meta.data)[seurat_data@meta.data$annotation == "CD8T"])
seurat_Mono_Macro <- subset(seurat_data, cells = rownames(seurat_data@meta.data)[seurat_data@meta.data$annotation == "Mono_Macro"])
seurat_Neutrophils <- subset(seurat_data, cells = rownames(seurat_data@meta.data)[seurat_data@meta.data$annotation == "Neutrophils"])
seurat_NK <- subset(seurat_data, cells = rownames(seurat_data@meta.data)[seurat_data@meta.data$annotation == "NK"])
seurat_B <- subset(seurat_data, cells = rownames(seurat_data@meta.data)[seurat_data@meta.data$annotation == "B"])
saveRDS(seurat_CD8T, file = "seurat_CD8T.rds")
saveRDS(seurat_Mono_Macro, file = "seurat_Mono_Macro.rds")
saveRDS(seurat_Neutrophils, file = "seurat_Neutrophils.rds")
saveRDS(seurat_NK, file = "seurat_NK.rds")
saveRDS(seurat_B, file = "seurat_B.rds")

gem_CD8T <- GetAssayData(seurat_CD8T, assay = "RNA", layer = "data")
write.csv(gem_CD8T, "gem_CD8T.csv", row.names = FALSE)
gem_Mono_Macro <- GetAssayData(seurat_Mono_Macro, assay = "RNA", layer = "data")
write.csv(Mono_Macro_data, "gem_Mono_Macro.csv", row.names = FALSE)
gem_Neutrophils <- GetAssayData(seurat_Neutrophils, assay = "RNA", layer = "data")
write.csv(gem_Neutrophils, "gem_Neutrophils.csv", row.names = FALSE)
gem_NK <- GetAssayData(seurat_NK, assay = "RNA", layer = "data")
write.csv(gem_NK, "gem_NK.csv", row.names = FALSE)
gem_B <- GetAssayData(seurat_B, assay = "RNA", layer = "data")
write.csv(gem_B, "gem_B.csv", row.names = FALSE)